# EARIN Exercise 5
#### Deininger Johannes, Hari Shankar Sasikumar Kolamannathodiyil



In [1190]:
import numpy as np
import pandas as pd
import random

iris=pd.read_csv('../input/irisdatajohannes/Iris.csv')
iris = iris.sample(frac=1).reset_index(drop=True) # Shuffle

X = iris[['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']]
X = np.array(X)
X[:5]

from sklearn.preprocessing import OneHotEncoder
oneHot = OneHotEncoder(sparse=False)

Y = iris.Species
Y = oneHot.fit_transform(np.array(Y).reshape(-1, 1))
Y[:5]

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2,random_state = 42)
# X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2,random_state = 42)

In [1191]:
def NN(X_train, Y_train, X_test=None, Y_test=None, epochs=10, nodes=[], lr=0.15, batch = 20):
    hidden_layers = len(nodes) - 1   
    lay, w = len(nodes), []
    for i in range(1, lay):
        w_ = [[0.1 for k in range(nodes[i-1] + 1)]
              for j in range(nodes[i])]
        w.append(np.matrix(w_))

    for epoch in range(1, epochs+1):
        w = Train(X_train, Y_train, lr, w, batch)

        if(epoch % 50 == 0):
            print("Epoch {}".format(epoch))
            print("Training Accuracy:{}".format(Accuracy(X_train, Y_train, w)))
            if X_test.any():
                print("Testing Accuracy:{}".format(Accuracy(X_test, Y_test, w)))
            
    return w

In [1192]:
def FwdProp(x, w, lay):
    a, lay_in = [x], x
    for j in range(lay):
       # print(j, layer_input.shape, weights[j].T.shape)
        a_ = Sigm(np.dot(lay_in, w[j].T))
        a.append(a_)
        lay_in = np.append(1, a_) # Augment with bias
    
    return a

In [1193]:
def BackProp(y, a, w, lay):
    outFin = a[-1]
    er = np.matrix(y - outFin) # Error at output
    #er = np.matrix(np.divide(y- outFin,np.multiply(outFin,1-outFin)))
    
    for j in range(lay, 0, -1):
        currAct = a[j]
        if(j > 1):
            # Augment previous activation
            prevAct = np.append(1, a[j-1])

        else:
            # First hidden layer, prevActivation is input (without bias)
            prevAct = a[0]
        
        delta = np.multiply(er, dSigm(currAct))
        w[j-1] += lr * np.multiply(delta.T, prevAct)

        w_ = np.delete(w[j-1], [0], axis=1) # Remove bias from weights
        er = np.dot(delta, w_) # Calculate error for current layer
    
    return w

In [1194]:
def Train(X, Y, lr, w, batch):
    lay = len(w)
    x_sub, y_sub = zip(*random.sample(list(zip(X,Y)),batch))
    for i in range(len(x_sub)):
        x, y = x_sub[i], y_sub[i]
        x = np.matrix(np.append(1, x)) # Augment feature vector
        
        a = FwdProp(x, w, lay)
        w = BackProp(y, a, w, lay)

    return w

In [1195]:
def Sigm(x):
    return 1 / (1 + np.exp(-x))

def dSigm(x):
    return np.multiply(x, 1-x)

def relu(x):
    print(np.maximum(0,x))
    return np.maximum(0,x)

def drelu(x):
    return (x > 0) * 1


In [1196]:
def Predict(item, w):
    lay = len(w)
    item = np.append(1, item) # Augment feature vector
    
    ##_Forward Propagation_##
    a = FwdProp(item, w, lay)
    
    outFin = a[-1].A1
    idx = maxAct(outFin)

    # Initialize prediction vector to zeros
    y = [0 for i in range(len(outFin))]
    y[idx] = 1  # Set guessed class to 1

    return y # Return prediction vector


def maxAct(out):
    """Find max activation in output"""
    m, idx = out[0], 0
    for i in range(1, len(out)):
        if(out[i] > m):
            m, idx = out[i], i
    
    return idx

In [1197]:
def Accuracy(X, Y, w):
    """Run set through network, find overall accuracy"""
    true = 0

    for i in range(len(X)):
        x, y = X[i], list(Y[i])
        pred = Predict(x, w)

        if(y == pred):
            # Guessed correctly
            true += 1

    return true*100 / len(X)

In [1198]:
f = len(X[0]) # Number of features
o = len(Y[0]) # Number of outputs / classes

layers = [f,10,o] # Number of nodes in layers
lr, epochs, batch = 0.1, 300, 25

weights = NN(X_train, Y_train, X_test, Y_test, epochs=epochs, nodes=layers, lr=lr, batch=batch);

Epoch 50
Training Accuracy:35.833333333333336
Testing Accuracy:23.333333333333332
Epoch 100
Training Accuracy:64.16666666666667
Testing Accuracy:76.66666666666667
Epoch 150
Training Accuracy:80.0
Testing Accuracy:90.0
Epoch 200
Training Accuracy:98.33333333333333
Testing Accuracy:96.66666666666667
Epoch 250
Training Accuracy:92.5
Testing Accuracy:93.33333333333333
Epoch 300
Training Accuracy:99.16666666666667
Testing Accuracy:93.33333333333333
